Robot Dynamic Model Identification
=======
---

# 1 - Praparing work

## Import libraries

In [30]:
# enable auto-reload modules
%load_ext autoreload
%autoreload 2
#interactive plotting in separate window
%matplotlib qt 
#%matplotlib inline

from sympy import init_printing
init_printing()
    
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 8]

import numpy as np
import sympy
from robot_def import RobotDef
from kinematics import Geometry
from dynamics import Dynamics
from trajectory_optimization import TrajOptimizer
from trajectory_optimization import TrajPlotter
from utils import new_sym
from utils import save_data, load_data
import time

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Folder dir for saving and loading files

In [31]:
# model_name = 'test_psm_long2'
# model_name ='test'
# model_name = 'mtm_3links'
# model_name = 'mtm'
# model_name = 'mtm_spring'
model_name = 'mtm_2spring'
# model_name = 'mtm_3links_parallel'
# model_name = 'mtm_4links_parallel'

model_folder = 'data/' + model_name + '/model/'

---
# 2 - Robot modelling

## Create joint variables and define their relations

## Robot geometry definition in following order

| Joint number | prev link | succ links | $a$ | $\alpha$ | $d$ | $\theta$ | link inertia | motor inertia | friction |
|--------------|-----------|------------|-----|----------|-----|----------|--------------|---------------|---------|

In [32]:
q0, q1, q2, q3, q4, q5, q6, q7, q8, q9, q10 = new_sym('q:11')
_pi = sympy.pi

dh = []
springs = []
friction_type = []

if model_name =='test':
    dh = [(0,   -1, [1],    0,     0,      0,        0,        False, False, False),
          (1,   0,  [2,8],  0,     0,      -0.21537, q1,       True,  False, False),
          (2,   1,  [3],    0,     -_pi/2, 0,        q2+_pi/2, True,  False, False)]
    q = q1
    r_s = 0.0075
    h_s = 0.1035
    l_r = 0.0613
    q_o = 23.0/180*_pi
    l = sympy.sqrt(r_s**2 + h_s**2 - 2*r_s*h_s*sympy.cos(_pi + q_o - q))
    d_l = l - l_r
    r_f = r_s * h_s * sympy.sin(_pi + q_o - q) / l
    tau_s = r_f * d_l # * 27.86
    springs = [tau_s]
    friction_type = ['Coulomb', 'viscous', 'offset']
elif model_name == 'mtm':
    # define paralelogram coordinate relation
    q3 = -q2 + q8
    q9 = -q8 + q2
    q10 = q8 - q2
    
    dh = [(0,   -1, [1],    0,     0,      0,        0,        False, False, False),
          (1,   0,  [2,8],  0,     0,      -0.21537, q1,       True,  False, True),
          (2,   1,  [3],    0,     -_pi/2, 0,        q2+_pi/2, True,  False, True),
          (3,   2,  [4],    0.279, 0,      0,        q3+_pi/2, True,  False, True),
          (4,   3,  [5],    0.365, -_pi/2, 0.151,    q4,       True,  False, True),
          (5,   4,  [6],    0,     _pi/2,  0,        q5,       True,  False, True),
          (6,   5,  [7],    0,     -_pi/2, 0,        q6+_pi/2, True,  False, True),
          (7,   6,  [],     0,     -_pi/2, 0,        q7+_pi,   True,  False, True),
          (8,   1,  [9],    0,     -_pi/2, 0,        q8+_pi,   True,  False, True),
          (9,   8,  [10],   0.1,   0,      0,        q9-_pi/2, True,  False, True),
          (10,  9,  [],     0.279, 0,      0,        q10,      False, False, True)]

    q = q5
    r_s = 0.0075
    h_s = 0.1035
    l_r = 0.0613
    q_o = 23.0/180.0*_pi
    l = sympy.sqrt(r_s**2 + h_s**2 - 2*r_s*h_s*sympy.cos(_pi + q_o - q))
    d_l = l - l_r
    r_f = r_s * h_s * sympy.sin(_pi + q_o - q) / l
    tau_s = r_f * d_l # * 27.86
    #springs = [tau_s]
    
    friction_type = ['Coulomb', 'viscous', 'offset']
    print("MTM")
elif model_name == 'mtm_spring':
    # define paralelogram coordinate relation
    q3 = -q2 + q8
    q9 = -q8 + q2
    q10 = q8 - q2
    
    dh = [(0,   -1, [1],    0,     0,      0,        0,        False, False, False),
          (1,   0,  [2,8],  0,     0,      -0.21537, q1,       True,  False, True),
          (2,   1,  [3],    0,     -_pi/2, 0,        q2+_pi/2, True,  False, True),
          (3,   2,  [4],    0.279, 0,      0,        q3+_pi/2, True,  False, True),
          (4,   3,  [5],    0.365, -_pi/2, 0.151,    q4,       True,  False, True),
          (5,   4,  [6],    0,     _pi/2,  0,        q5,       True,  False, True),
          (6,   5,  [7],    0,     -_pi/2, 0,        q6+_pi/2, True,  False, True),
          (7,   6,  [],     0,     -_pi/2, 0,        q7+_pi,   True,  False, True),
          (8,   1,  [9],    0,     -_pi/2, 0,        q8+_pi,   True,  False, True),
          (9,   8,  [10],   0.1,   0,      0,        q9-_pi/2, True,  False, True),
          (10,  9,  [],     0.279, 0,      0,        q10,      False, False, True)]

    q = q5
    r_s = 0.0075
    h_s = 0.1035
    l_r = 0.0613
    q_o = 23.0/180.0*_pi
    l = sympy.sqrt(r_s**2 + h_s**2 - 2*r_s*h_s*sympy.cos(_pi + q_o - q))
    d_l = l - l_r
    r_f = r_s * h_s * sympy.sin(_pi + q_o - q) / l
    tau_s = r_f * d_l # * 27.86
    springs = [tau_s]
    
    friction_type = ['Coulomb', 'viscous', 'offset']
    print(model_name)
elif model_name == 'mtm_2spring':
    # define paralelogram coordinate relation
    q3 = -q2 + q8
    q9 = -q8 + q2
    q10 = q8 - q2
    
    dh = [(0,   -1, [1],    0,     0,      0,        0,        False, False, False),
          (1,   0,  [2,8],  0,     0,      -0.21537, q1,       True,  False, True),
          (2,   1,  [3],    0,     -_pi/2, 0,        q2+_pi/2, True,  False, True),
          (3,   2,  [4],    0.279, 0,      0,        q3+_pi/2, True,  False, True),
          (4,   3,  [5],    0.365, -_pi/2, 0.151,    q4,       True,  False, True),
          (5,   4,  [6],    0,     _pi/2,  0,        q5,       True,  False, True),
          (6,   5,  [7],    0,     -_pi/2, 0,        q6+_pi/2, True,  False, True),
          (7,   6,  [],     0,     -_pi/2, 0,        q7+_pi,   True,  False, True),
          (8,   1,  [9],    0,     -_pi/2, 0,        q8+_pi,   True,  False, True),
          (9,   8,  [10],   0.1,   0,      0,        q9-_pi/2, True,  False, True),
          (10,  9,  [],     0.279, 0,      0,        q10,      False, False, True)]

    q = q5
    r_s = 0.0075
    h_s = 0.1035
    l_r = 0.0613
    q_o = 23.0/180.0*_pi
    l = sympy.sqrt(r_s**2 + h_s**2 - 2*r_s*h_s*sympy.cos(_pi + q_o - q))
    d_l = l - l_r
    r_f = r_s * h_s * sympy.sin(_pi + q_o - q) / l
    tau_s = r_f * d_l # * 27.86
    springs = [tau_s, -q4]
    
    friction_type = ['Coulomb', 'viscous', 'offset']
    print(model_name)
elif model_name == 'mtm_3links_parallel':
    # define paralelogram coordinate relation
    q3 = -q2 + q4
    q5 = -q4 + q2
    q6 = q4 - q2
    
    dh = [(0,   -1, [1],    0,     0,      0,        0,        False, False, False),
          (1,   0,  [2,4],  0,     0,      -0.21537, q1,       True,  False, True),
          (2,   1,  [3],    0,     -_pi/2, 0,        q2+_pi/2, True,  False, True),
          (3,   2,  [4],    0.279, 0,      0,        q3+_pi/2, True,  False, True),
          (4,   1,  [5],    0,     -_pi/2, 0,        q4+_pi,   True,  False, True),
          (5,   4,  [6],   0.1,   0,      0,        q5-_pi/2, True,  False, True),
          (6,   5,  [],     0.279, 0,      0,        q6,      False, False, True)]
    friction_type = ['Coulomb', 'viscous']
    #springs = [q1, q2, q3]
elif model_name == 'mtm_4links_parallel':
    # define paralelogram coordinate relation
    q3 = -q2 + q5
    q6 = -q5 + q2
    q7 = q5 - q2
    
    dh = [(0,   -1, [1],    0,     0,      0,        0,        False, False, False),
          (1,   0,  [2,5],  0,     0,      -0.21537, q1,       True,  False, True),
          (2,   1,  [3],    0,     -_pi/2, 0,        q2+_pi/2, True,  False, True),
          (3,   2,  [4],    0.279, 0,      0,        q3+_pi/2, True,  False, True),
          (4,   3,  [0],    0.365, -_pi/2, 0.151,    q4,       True,  False, True),
          (5,   1,  [6],     0,     -_pi/2, 0,        q5+_pi,   True,  False, True),
          (6,   5,  [7],   0.1,   0,      0,        q6-_pi/2, True,  False, True),
          (7,   6,  [],     0.279, 0,      0,        q7,      False, False, True)]
    friction_type = ['Coulomb', 'viscous']
elif model_name == 'mtm_3links':
    dh = [(0,   -1, [1],    0,      0,      0,          0,         False, False, False),
          (1,   0,  [2],    0,      0,      -0.21537,   q1,        True,  False, True),
          (2,   1,  [3],    0,      -_pi/2, 0,          q2+_pi/2,  True,  False, True),
          (3,   2,  [4],    0.279,  0,      0,          q3+_pi/2,  True,  True,  True)]
    # springs = [q1]
    friction_type = ['Coulomb', 'viscous']
elif (model_name == 'test_psm'):
    beta = 0.2908
    beta2 = 0.0767 + beta
    delta = 0.6025
    x = [None]*10
    x[0] = (0,   -1, [1],        0,              0,     0,                       0,      False,  False, False) # Base
    x[1] = (1,   0,  [2],        0,     sympy.pi/2,     0,           q1+sympy.pi/2,       True,  False,  True) # Yaw
    x[2] = (2,   1,  [3, 4],        0, -sympy.pi/2,     0,              sympy.pi/2,       False, False, False) # -- Intermediate
    x[3] = (3,   2,  [5, 6],  -0.0296,           0,     0,    q2-beta - sympy.pi/2,        True,  False, True) # Pitch Back
    x[4] = (4,   2,  [] ,     0.0664 ,           0,     0,    q2-beta - sympy.pi/2,        True,  False, True) # Pitch Front                                    )
    x[5] = (5,   2,  [7],     -0.0296,           0,     0,   q2-beta2 - sympy.pi/2,        False,  False, False)    # Intermediate
    x[6] = (6,   3,  [8],       0.150,           0,     0,    -q2+beta+sympy.pi/2,      True,  False,  True)    # Pitch Bottom
    x[7] = (7,   5,  [],       0.1842,           0,     0,    -q2+beta2+sympy.pi/2,     True,  False,  True)    # Pitch Top
    x[8] = (8,   6,  [9],       0.516,           0,     0,                      q2,     True,  False,  True)    # Pitch End
    x[9] = (9,   8,  [10],      0.043, -sympy.pi/2,-.28809+q3,          sympy.pi/2,     True,  False, True)    # Main Insertion
    dh = x
    friction_type = ['Coulomb', 'viscous', 'offset']              
elif (model_name == 'test_psm_long2'):
    beta = 0.2908
    beta2 = 0.0767 + beta
    delta = 0.6025
    
    x = [None]*17
    x[0]  = ( 0,  -1,  [1],          0,           0,     0,                       0, False, False, False) # Base
    x[1]  = ( 1,   0,  [2],          0,  sympy.pi/2,     0,           q1+sympy.pi/2,  True, False,  True) # Yaw
    x[2]  = ( 2,   1,  [3, 4, 5],       0, -sympy.pi/2,     0,              sympy.pi/2, False, False, False) # -- Intermediate
    x[3]  = ( 3,   2,  [6, 10], -0.0296,       0,     0,    q2-beta - sympy.pi/2,  True, False,  True) # Pitch Back
    x[4]  = ( 4,   2,  [] ,     0.0664,           0,     0,    q2-beta - sympy.pi/2,  True, False,  True) # Pitch Front                                    )
    x[5]  = ( 5,   2,  [7],    -0.0296,           0,     0,   q2-beta2 - sympy.pi/2, False, False, False)    # Intermediate
    x[6]  = ( 6,   3,  [8],      0.150,           0,     0,     -q2+beta+sympy.pi/2,  True, False,  True)    # Pitch Bottom
    x[7]  = ( 7,   5,   [],     0.1842,           0,     0,    -q2+beta2+sympy.pi/2,  True, False,  True)    # Pitch Top
    x[8]  = ( 8,   6,  [9],      0.516,           0,     0,                      q2,  True, False,  True)    # Pitch End
    x[9]  = ( 9,   8, [12],      0.043, -sympy.pi/2,-.28809+q3,          sympy.pi/2,  True, False,  True)    # Main Insertion
    x[10] = (10,   3, [11],          0,           0,     0,         beta+sympy.pi/2, False, False, False) # Intermediate Counterweight
    x[11] = (11,  10,   [],       -0.1, +sympy.pi/2, delta*q3,                    0,  True, False,  True) # Counterweight
    
    x[12] = (12,   9, [13],          0,           0,0.4162,          -sympy.pi/2+q4, False,  True,  True) # Tool Roll 
    x[13] = (13,  12, [14],          0, -sympy.pi/2,     0,          -sympy.pi/2+q5, False,  True,  True) # Tool Pitch
    x[14] = (14,  13, [15,16],  0.0091, -sympy.pi/2,     0,          -sympy.pi/2+q6, False,  True,  True) # Tool Yaw
    x[15] = (15,  14, [],            0,           0,     0,                     -q7, False, False,  True) # Gripper1
    x[16] = (16,  15, [],            0,           0,     0,                      q7, False,  True,  True) # Gripper2
    dh = x
    friction_type = ['Coulomb', 'viscous', 'offset']
else:
    raise Exception("No robot name matches " + name)
print(friction_type)
robot_def = RobotDef(dh,
                     springs=springs,
                     dh_convention='mdh',
                     friction_type=friction_type)

mtm_2spring
['Coulomb', 'viscous', 'offset']


In [33]:
robot_def.bary_params

## Create kinematics chain

In [34]:
geom = Geometry(robot_def)

In [35]:
angle = [0, 0, 0, 0, 0, 0, 0]
geom.draw_geom(angle)

## Create dynamics

In [36]:
%time dyn = Dynamics(robot_def, geom)

Calculating Lagrangian...
Calculating the link kinetic energy of 1/10
Calculating the link kinetic energy of 2/10
Calculating the link kinetic energy of 3/10
Calculating the link kinetic energy of 4/10
Calculating the link kinetic energy of 5/10
Calculating the link kinetic energy of 6/10
Calculating the link kinetic energy of 7/10
Calculating the link kinetic energy of 8/10
Calculating the link kinetic energy of 9/10
Calculating joint torques...
tau of q1
tau of q2
tau of q8
tau of q4
tau of q5
tau of q6
tau of q7
Adding frictions, motor rotor inertia and springs...
Calculating gregressor...
('input_vars', (q1, q2, q8, q4, q5, q6, q7, dq1, dq2, dq8, dq4, dq5, dq6, dq7, ddq1, ddq2, ddq8, ddq4, ddq5, ddq6, ddq7))
Calculating M, C and G...
Calculating base parameter...
[ 79  80  82  66  69  67  53  83  85  78  84  71  72  56  81  58  59  54
  40  45  46  65  47  41  43 110  32 111  57  14  55  92  44  37  30  39
  17  28  15  16  42  97  98  63  76  29  89  50  18 121  11  24 102  38
   

In [37]:
robot_def.bary_params

In [38]:
sympy.Matrix(dyn.base_param)

⎡                                                                  1.0⋅L7xy   
⎢                                                                             
⎢                                                                  1.0⋅L7xz   
⎢                                                                             
⎢                                                                  1.0⋅L7yz   
⎢                                                                             
⎢                                                                  1.0⋅L6xy   
⎢                                                                             
⎢                                                                  1.0⋅L6yz   
⎢                                                                             
⎢                                                                  1.0⋅L6xz   
⎢                                                                             
⎢                                                   

In [39]:
from data import RobotModel

# Data to be saved
robot_model = RobotModel(dyn)

# Save
save_data(model_folder, model_name, robot_model)
print('Saved {} parameters'.format(len(robot_model.base_param)))

Saved 71 parameters


---
# 3 - Excitation trajectory optimization

In [53]:
# model_name = 'test_psm_long2'
# model_name ='test'
# model_name = 'mtm_3links'
# model_name = 'mtm'
#model_name = 'mtm_spring'
model_name = 'mtm_2spring'
#model_name = 'mtm_3links_parallel'
# model_name = 'mtm_4links_parallel'

Load Data

In [54]:
from utils import load_data
model_folder = 'data/' + model_name + '/model/'
robot_model = load_data(model_folder, model_name)

## Create trajectory optimizer object, with H matrix, foourier order, base frequency, joint constraints and cartisian constraints as input

In [55]:
#trajectory_name = 'one'
trajectory_name = 'two'
# trajectory_name = 'two_normalized'

In [56]:
from numpy import deg2rad

optimal_trajectory_folder = 'data/optimal_trajectory/'
trajectory_folder = 'data/' + model_name +'/optimal_trajectory/'

base_freq = 0.1
fourier_order = 6

joint_constraints = []
cartesian_constraints = []

if model_name == 'mtm' or model_name == 'mtm_spring' or model_name == 'mtm_2spring':
#     joint_constraints = [(q1, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q2, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q4, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q5, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q6, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q7, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q8, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi)]
#     joint_constraints = [(q1, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q2, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q4, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q5, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q8, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi)]

    joint_constraints = [(q1, deg2rad(-57), deg2rad(29), deg2rad(-160), deg2rad(160)),
                         (q2, deg2rad(-10), deg2rad(60), deg2rad(-180), deg2rad(180)),
                         (q3, deg2rad(-30), deg2rad(30), deg2rad(-180), deg2rad(180)),
                         (q4, deg2rad(-40), deg2rad(195), -6*np.pi, 6*np.pi),
                         (q5, deg2rad(-87), deg2rad(180), -6*np.pi, 6*np.pi),
                         (q6, deg2rad(-40), deg2rad(38), -6*np.pi, 6*np.pi),
                         (q7, deg2rad(-460), deg2rad(450), -10*np.pi, 10*np.pi),
                         (q8, deg2rad(-9), deg2rad(39), deg2rad(-150), deg2rad(150))]

#     cartesian_constraints = [(3,'max',1,0.4,1),
#                              (3,'min',-1,-1,-1)]
elif model_name == 'mtm_3links':
    joint_constraints = [(q1, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
                         (q2, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
                         (q3, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi)]
    
    cartesian_constraints = [(1,'max',1,0.4,1),
                             (1,'min',-1,-1,-1),
                             (2,'max',1,0.4,1),
                             (2,'min',-1,-1,-1)]
    
    # MTM STUFF
#jnt_cnst = [(q1, -0.698132, 1.13446, -2, 2),(q2, -0.261799, 0.872665, -2, 2), (q3,-0.172665, 0.610865, -2, 2)]
#jnt_cnst = [(q1, -1.5, 1.5, -2, 2),(q2, -1.5, 1.5, -2, 2), (q3,-1.5, 1.5, -2, 2)]    

#crt_cnst = [(4, 'max', -0.283, 0.279, -0.25137), (4, 'min', -0.5, -0.321, -0.39137)]
#crt_cnst = [(3, 'max', -0.300, 0.3, -0.26137), (3, 'min', -0.5, -0.45, -0.53137)]
elif model_name == 'mtm_3links_parallel' or model_name == 'mtm_3links_parallel_offset':
    joint_constraints = [(q1, deg2rad(-60), deg2rad(32), deg2rad(-180), deg2rad(180)),
                         (q2, deg2rad(-12), deg2rad(62), deg2rad(-180), deg2rad(180)),
                         (q3, deg2rad(-32), deg2rad(32), deg2rad(-180), deg2rad(180)),
                         (q4, deg2rad(-9), deg2rad(39), deg2rad(-360), deg2rad(360))]
elif model_name == 'mtm_4links_parallel':
    joint_constraints = [(q1, deg2rad(-60), deg2rad(32), deg2rad(-180), deg2rad(180)),
                         (q2, deg2rad(-12), deg2rad(62), deg2rad(-180), deg2rad(180)),
                         (q3, deg2rad(-32), deg2rad(32), deg2rad(-180), deg2rad(180)),
                         (q4, deg2rad(-90), deg2rad(195), -2*np.pi, 2*np.pi),
                         (q5, deg2rad(-9), deg2rad(39), deg2rad(-360), deg2rad(360))]
elif model_name == 'psm':
    pass
elif model_name == 'test_psm_long2':
    joint_constraints = [(q1, -1.4, 1.4, -2, 2),
                         (q2, -0.5, 0.8, -2, 2),
                         (q3,0.05,0.23,-0.4,0.4),
                         (q4,-1.5,1.5,-0.4,0.4),
                         (q5,-1.5,1.5,-0.4,0.4), 
                         (q6,-1.5,1.5,-0.4,0.4),
                         (q7,0.05, 0.75,-0.4,0.4)]
else:
    raise Exception("No robot name matches " + model_name)
    
traj_optimizer = TrajOptimizer(robot_model, fourier_order, base_freq,
                               joint_constraints=joint_constraints,
                               cartesian_constraints = cartesian_constraints)
                                                                                                 
                                                                                                                     
traj_optimizer.optimize()


joint constraint number: 8
cartesian constraint number: 0
constraint number: 32
frames_constrained: []
cond: 87.7844515396
cond: 87.7844515396
cond: 87.7844517411
cond: 87.7844496366
cond: 87.7844519936
cond: 87.7844531949
cond: 87.7844542367
cond: 87.784454835
cond: 87.7844560178
cond: 87.7844564924
cond: 87.7844532542
cond: 87.7844554121
cond: 87.7844533403
cond: 87.7844549893
cond: 87.7844552382
cond: 87.784447414
cond: 87.784452367
cond: 87.7844490537
cond: 87.7844495685
cond: 87.7844490945
cond: 87.7844486906
cond: 87.7844516008
cond: 87.7844499621
cond: 87.7844526465
cond: 87.784456904
cond: 87.7844498346
cond: 87.7844581489
cond: 87.7844315338
cond: 87.7844581732
cond: 87.7844507421
cond: 87.7844520152
cond: 87.7844523606
cond: 87.7844510574
cond: 87.7844530988
cond: 87.7844883861
cond: 87.7844715255
cond: 87.7844689695
cond: 87.7844443691
cond: 87.7844611664
cond: 87.7844343588
cond: 87.7844333033
cond: 87.7844423206
cond: 87.7844445764
cond: 87.7844457826
cond: 87.7844473724
c

cond: 102.153159364
cond: 102.153105896
cond: 102.153154243
cond: 102.153123835
cond: 102.153115341
cond: 102.153106289
cond: 102.153126132
cond: 102.15311404
cond: 102.153113324
cond: 102.15311462
cond: 102.153112355
cond: 102.153132109
cond: 102.153119124
cond: 102.153113858
cond: 102.153119583
cond: 102.153117164
cond: 102.153110415
cond: 102.153115464
cond: 102.15311568
cond: 102.153118487
cond: 102.153117882
cond: 102.153117527
cond: 102.153114414
cond: 102.153114982
cond: 102.153114475
cond: 102.153114109
cond: 102.153114524
cond: 102.153113254
cond: 102.15311388
cond: 102.15311458
cond: 102.153117758
cond: 102.153114238
cond: 102.153112388
cond: 102.153114853
cond: 102.15311575
cond: 102.153116164
cond: 102.153116226
cond: 102.153116274
cond: 102.15311626
cond: 102.153116311
cond: 102.153117951
cond: 102.153115792
cond: 102.153115769
cond: 102.153115556
cond: 102.15311733
cond: 102.153117011
cond: 102.153116489
cond: 102.153116051
cond: 102.153115701
cond: 102.153115607
cond: 10

cond: 275.612068092
cond: 275.61206954
cond: 275.61206858
cond: 275.612067164
cond: 275.612066402
cond: 275.61206744
cond: 275.612063153
cond: 275.612070254
cond: 275.61206872
cond: 275.612068018
cond: 275.612066442
cond: 275.612067279
cond: 336.026023398
cond: 261.409540646
cond: 261.409540646
cond: 261.409549074
cond: 261.409487847
cond: 261.409539539
cond: 261.409518365
cond: 261.409517499
cond: 261.409618976
cond: 261.409554288
cond: 261.409535164
cond: 261.409538232
cond: 261.409508124
cond: 261.40953245
cond: 261.409551244
cond: 261.409497561
cond: 261.409499097
cond: 261.409442462
cond: 261.409581225
cond: 261.409501267
cond: 261.409527322
cond: 261.409542918
cond: 261.409939604
cond: 261.409461324
cond: 261.409372756
cond: 261.409687133
cond: 261.409375805
cond: 261.409470374
cond: 261.409571368
cond: 261.409295756
cond: 261.409495281
cond: 261.409522213
cond: 261.409528151
cond: 261.409615957
cond: 261.409575629
cond: 261.408992217
cond: 261.409593444
cond: 261.409745354
cond:

cond: 250.569770223
cond: 250.570050579
cond: 250.569744341
cond: 250.569814386
cond: 250.569922294
cond: 250.569827902
cond: 250.569891095
cond: 250.569853305
cond: 250.569932173
cond: 250.569962802
cond: 250.569946868
cond: 250.569361641
cond: 250.569949365
cond: 250.570082413
cond: 250.569820793
cond: 250.570037509
cond: 250.569923233
cond: 250.569900291
cond: 250.569895346
cond: 250.569900398
cond: 250.569899557
cond: 250.569898276
cond: 250.569900353
cond: 250.569897972
cond: 250.56990594
cond: 250.569897399
cond: 250.569895545
cond: 250.569902342
cond: 250.569901006
cond: 250.569899328
cond: 250.569900107
cond: 250.569898206
cond: 250.569899373
cond: 250.569896831
cond: 250.569899703
cond: 250.569898463
cond: 250.569898284
cond: 250.569895574
cond: 250.569896463
cond: 250.569897332
cond: 250.569896609
cond: 250.569899792
cond: 250.569897269
cond: 250.569900066
cond: 250.569898922
cond: 250.569899735
cond: 250.56990018
cond: 250.569898232
cond: 250.569899236
cond: 250.56989947
con

cond: 234.27223559
cond: 234.272235381
cond: 234.272234433
cond: 234.272235074
cond: 234.272234171
cond: 234.272234826
cond: 234.272231695
cond: 234.272235385
cond: 234.272231972
cond: 234.272236164
cond: 234.272235094
cond: 234.272231959
cond: 234.272234986
cond: 234.272229423
cond: 234.272234297
cond: 234.27223765
cond: 234.272232798
cond: 234.272231326
cond: 234.272234034
cond: 234.27223624
cond: 234.272233734
cond: 234.272234162
cond: 234.27223454
cond: 234.272233921
cond: 234.272233903
cond: 234.272229238
cond: 234.272234058
cond: 234.272234379
cond: 234.272233624
cond: 234.272234469
cond: 234.272234029
cond: 234.343636932
cond: 232.94114444
cond: 231.702072319
cond: 231.702072319
cond: 231.702087118
cond: 231.702035142
cond: 231.702081852
cond: 231.702063318
cond: 231.702048317
cond: 231.702115981
cond: 231.70207067
cond: 231.702071475
cond: 231.702071085
cond: 231.702058551
cond: 231.702071693
cond: 231.702089396
cond: 231.702054952
cond: 231.70201024
cond: 231.702040383
cond: 2

cond: 213.729766371
cond: 213.729701343
cond: 213.729704085
cond: 213.729698007
cond: 213.729698955
cond: 213.729715509
cond: 213.729718514
cond: 213.729721784
cond: 213.729693203
cond: 213.729610241
cond: 213.729788728
cond: 213.729684764
cond: 213.729673019
cond: 213.729683545
cond: 213.729995033
cond: 213.729670251
cond: 213.729601717
cond: 213.729819878
cond: 213.72959575
cond: 213.729613844
cond: 213.729731131
cond: 213.729694544
cond: 213.729833017
cond: 213.729641882
cond: 213.729704918
cond: 213.72973132
cond: 213.72974868
cond: 213.729300441
cond: 213.729793491
cond: 213.729850719
cond: 213.72965457
cond: 213.729799607
cond: 213.729756075
cond: 213.729701746
cond: 213.729706969
cond: 213.729704427
cond: 213.729704509
cond: 213.729703161
cond: 213.729701933
cond: 213.729704573
cond: 213.729707721
cond: 213.729703536
cond: 213.729705667
cond: 213.729709534
cond: 213.729704095
cond: 213.729702782
cond: 213.729701015
cond: 213.729695231
cond: 213.729703638
cond: 213.729702006
cond

cond: 207.935326015
cond: 207.935327113
cond: 207.935325314
cond: 207.935327804
cond: 207.93532535
cond: 207.93532686
cond: 207.935327504
cond: 207.935326844
cond: 207.935325805
cond: 207.935325473
cond: 207.93532785
cond: 207.93532605
cond: 207.935326322
cond: 207.93532682
cond: 207.935325901
cond: 207.935326555
cond: 207.935325371
cond: 207.935327397
cond: 207.935327237
cond: 207.935326036
cond: 207.935326194
cond: 207.935325849
cond: 207.935326927
cond: 207.93532682
cond: 207.935325608
cond: 207.935327613
cond: 207.935327582
cond: 207.935326458
cond: 207.935326845
cond: 207.935325243
cond: 207.935327598
cond: 207.935326516
cond: 207.935327099
cond: 207.935326081
cond: 207.935326103
cond: 207.935326459
cond: 207.935326366
cond: 207.935326814
cond: 207.935326495
cond: 207.935326312
cond: 207.935326528
cond: 207.935326917
cond: 207.935325991
cond: 207.9353269
cond: 207.935326349
cond: 207.935326005
cond: 207.935326171
cond: 207.935326192
cond: 219.946878205
cond: 207.846703211
cond: 20

cond: 206.73606459
cond: 206.73606443
cond: 206.736064476
cond: 206.736064132
cond: 206.73606407
cond: 206.736064597
cond: 206.736064591
cond: 206.736064814
cond: 206.736065053
cond: 206.736064823
cond: 206.736064522
cond: 211.787988294
cond: 206.98904837
cond: 206.550193995
cond: 206.550193995
cond: 206.550208339
cond: 206.550163236
cond: 206.550188282
cond: 206.550170842
cond: 206.550172297
cond: 206.550249723
cond: 206.550189968
cond: 206.550184724
cond: 206.550190673
cond: 206.550189158
cond: 206.550206213
cond: 206.550202124
cond: 206.550193829
cond: 206.550198446
cond: 206.55011307
cond: 206.550266843
cond: 206.550155662
cond: 206.550168912
cond: 206.550159207
cond: 206.550477876
cond: 206.550162678
cond: 206.550104748
cond: 206.55028072
cond: 206.550093849
cond: 206.550109584
cond: 206.550216122
cond: 206.550179595
cond: 206.550290641
cond: 206.550165778
cond: 206.550217907
cond: 206.550231387
cond: 206.550239384
cond: 206.54973271
cond: 206.550210445
cond: 206.550316647
cond: 2

cond: 206.315661891
cond: 206.315675643
cond: 206.315665205
cond: 206.315984669
cond: 206.315669342
cond: 206.31561073
cond: 206.315786188
cond: 206.315600005
cond: 206.315615242
cond: 206.315720912
cond: 206.315689509
cond: 206.31579719
cond: 206.315668376
cond: 206.315724198
cond: 206.315736759
cond: 206.315746111
cond: 206.315249056
cond: 206.315721823
cond: 206.315823791
cond: 206.315649751
cond: 206.315795203
cond: 206.315757469
cond: 206.315701045
cond: 206.315701545
cond: 206.315699418
cond: 206.315699786
cond: 206.315699895
cond: 206.315701088
cond: 206.315699784
cond: 206.315703436
cond: 206.315699724
cond: 206.315700535
cond: 206.315700261
cond: 206.315700485
cond: 206.315700207
cond: 206.315701217
cond: 206.315699997
cond: 206.315700861
cond: 206.315700371
cond: 206.315700701
cond: 206.315700306
cond: 206.315700219
cond: 206.315700046
cond: 206.315700121
cond: 206.315700303
cond: 206.31570068
cond: 206.315700678
cond: 206.315700225
cond: 206.315700562
cond: 206.315699317
con

cond: 206.277613951
cond: 206.277617563
cond: 206.277613832
cond: 206.277614715
cond: 206.277614438
cond: 206.277614654
cond: 206.277614368
cond: 206.277615321
cond: 206.277614164
cond: 206.277614991
cond: 206.277614541
cond: 206.277614863
cond: 206.277614452
cond: 206.277614379
cond: 206.277614277
cond: 206.277614312
cond: 206.277614463
cond: 206.277614821
cond: 206.277614807
cond: 206.277614376
cond: 206.277614736
cond: 206.277613611
cond: 206.277615175
cond: 206.277614212
cond: 206.277613658
cond: 206.277614395
cond: 206.277614551
cond: 206.277615089
cond: 206.277614513
cond: 206.277614711
cond: 206.277614469
cond: 206.277614632
cond: 206.277614336
cond: 206.27761451
cond: 206.277614819
cond: 206.27761462
cond: 206.277614585
cond: 206.277614492
cond: 206.277614301
cond: 206.27761432
cond: 206.277614796
cond: 206.27761461
cond: 206.277614752
cond: 206.277614866
cond: 206.277614628
cond: 206.277614474
cond: 237.959204286
cond: 206.634620077
cond: 206.661374607
cond: 206.760556888
cond

In [57]:
reg_norm_mat = traj_optimizer.calc_normalize_mat()

## Plot optimal excitation trajectory

In [46]:
traj_optimizer.calc_frame_traj()

In [50]:
traj_plotter = TrajPlotter(traj_optimizer.fourier_traj,traj_optimizer.frame_traj,
                           traj_optimizer.const_frame_ind, robot_model.coordinates)
traj_plotter.plot_desired_traj(traj_optimizer.x_result)

traj_plotter.plot_frame_traj(True)

## Save trajectory for later use

In [51]:
dof_order_bf_x_norm = (traj_optimizer.fourier_traj.dof, fourier_order,
                       base_freq, traj_optimizer.x_result, reg_norm_mat)
save_data(trajectory_folder, trajectory_name, dof_order_bf_x_norm)

freq = 500
tf = 35 # including 5 seconds' prestable time
traj_optimizer.make_traj_csv(trajectory_folder, trajectory_name, freq, tf)

In [52]:
from IPython.display import HTML, display
import tabulate

table = []
table.append(["joint", 'qo'] +
             ["a"+str(i+1) for i in range(fourier_order)] +
             ["b"+str(i+1) for i in range(fourier_order)])
for i in range(traj_optimizer.fourier_traj.dof):
    line = []
    line.append(robot_model.coordinates[i])
    line += np.round(traj_optimizer.x_result[i*(1 + fourier_order*2): (i+1)*(1 + fourier_order*2)], 4).tolist()
    table.append(line)

display(HTML(tabulate.tabulate(table, tablefmt='html')))

joint,qo,a1,a2,a3,a4,a5,a6,b1,b2,b3,b4,b5,b6
q1,-0.2305,0.0365,-0.0533,-0.0309,0.1137,0.0827,2.6281,-0.0007,0.0771,-0.1183,-0.0248,0.212,0.6409
q2,0.3439,0.0707,-0.1548,-0.1014,-0.2553,0.1854,0.387,0.0166,0.3863,0.051,0.8859,-0.1995,-0.9341
q8,0.308,-0.0115,-0.0084,0.0559,-0.1531,-0.3528,0.5565,-0.104,0.0024,-0.0558,0.2911,0.4162,-0.7182
q4,1.2666,-0.1077,0.0244,-0.1766,0.1829,0.6939,-1.7186,0.2642,1.0795,0.4964,-0.017,-2.3291,0.2692
q5,0.3422,-0.5249,-0.1741,-1.206,-0.9952,-0.3436,0.3237,-0.3805,-0.3327,0.5253,0.2745,0.2215,0.7684
q6,0.0342,-0.0772,0.0558,-0.0636,0.058,-0.1256,-0.5243,-0.0612,0.1768,0.0496,-0.291,-0.2854,-0.2442
q7,0.0344,-1.0205,1.2635,-3.1957,3.1248,5.571,0.5602,0.0419,-0.6761,2.1075,4.793,1.7796,9.7753


---
# Data processing

In [7]:
from identification import load_trajectory_data, diff_and_filt_data, plot_trajectory_data, plot_meas_pred_tau, gen_regressor

## Load data

In [8]:
# Names as Arguments
# model_name = 'test_psm_long'
# model_name = 'mtm_3links'
# model_name = 'mtm'
model_name = 'mtm_spring'
# model_name == 'mtm_3links_parallel'
# model_name == 'mtm_4links_parallel'

# trajectory_name = 'one'
# trajectory_name = 'two'
trajectory_name = 'one'

In [9]:
# Folders Standard
model_folder = 'data/' + model_name + '/model/'
trajectory_folder = 'data/' + model_name +'/optimal_trajectory/'
results_folder = 'data/' + model_name +'/measured_trajectory/'
results_data_file = results_folder + trajectory_name + '_results.csv'

# Load Results
dof, fourier_order, base_freq, traj_optimizer_result, reg_norm_mat = load_data(trajectory_folder, trajectory_name)


trajectory_sampling_rate = 500
#t, q_raw, tau_raw = load_trajectory_data(results_data_file, trajectory_sampling_rate)
t, q_raw, dq_raw, tau_raw = load_trajectory_data(results_data_file, trajectory_sampling_rate)
dof = q_raw.shape[1]

# Load Robot Model
robot_model = load_data(model_folder,model_name)

(<type 'numpy.ndarray'>, (14989, 21))


## Calculate filter cut-off frequency

In [10]:
fc_mult = 4.0
# this should be loaded from previously saved data
wf = 0.1
order = 6
fc = wf * order * fc_mult
print("cut frequency is {}".format(fc))

cut frequency is 2.4


## Get velocity and acceleration data and filter raw data

In [11]:
t_cut, q_f, dq_f, ddq_f, tau_f, q_raw_cut, tau_raw_cut = diff_and_filt_data(dof, 1.0/trajectory_sampling_rate, t,
                                                                        q_raw, dq_raw, tau_raw, fc, fc, fc, fc)
plot_trajectory_data(t_cut, q_raw_cut, q_f, dq_f, ddq_f, tau_raw_cut, tau_f)

q_raw shape: (14989, 7)
0
1
2
3
4
5
6
plot shape: 470


---
# Regression

## Ordinary Least Square (OLS)

### Generate regressor matrix for base parameters

In [12]:
base_param_num = robot_model.base_num
H_b_func = robot_model.H_b_func
W_b, tau_s = gen_regressor(base_param_num, H_b_func, q_f, dq_f, ddq_f, tau_f)

In [13]:
print(W_b.shape, tau_s.shape)
print np.linalg.cond(W_b)

((102123, 70), (102123,))
1272411.6409042422


In [14]:
xb_ols = np.linalg.lstsq(W_b, tau_s)[0]

/home/yan/anaconda2/envs/dynamics/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """Entry point for launching an IPython kernel.


In [15]:
#sympy.Matrix.hstack(sympy.Matrix(robot_model.base_param), sympy.Matrix(xb_ols))

from IPython.display import HTML, display
import tabulate

table = [["Base Parameter", "Value"]]

for i in range(robot_model.base_num):
    param_str = str(sympy.Matrix(robot_model.base_param)[i])
    max_disp_len = 50
    line = [param_str if len(param_str) <= max_disp_len else param_str[:max_disp_len] + '...', xb_ols[i]]
    #['%.7s ...'%b if len(str(b)) > 7 else str(b)
    table.append(line)

display(HTML(tabulate.tabulate(table, tablefmt='html')))

Base Parameter,Value
1.0*L7xy,-3.7606175214867072e-06
1.0*L7xz,-1.2450854463090131e-05
1.0*L7yz,-1.1660400486733329e-05
1.0*L6xy,0.0007151560568303215
1.0*L6xz,0.00015479673256722955
1.0*L6yz,-3.0292284948964937e-06
1.0*L6xx - 1.0*L6yy + 1.0*L7xx,-0.00014056381839007426
1.0*L7zz,-8.435077808115199e-06
1.0*l7x,0.00016686052118938116
1.0*l7y,0.000176728317155872


## Compare measured torque and predicted torque

### Generate predicted torque

In [16]:
tau_p = np.zeros(tau_f.shape)
tau_ps = W_b.dot(xb_ols)
for i in range(dof):
    tau_p[:, i] = tau_ps[i::dof]
    

### Evaluate regression

In [17]:
var_regression_error_ols = np.linalg.norm(tau_ps - tau_s) / (tau_ps.size - base_param_num)
print("variance of regression error using OLS:")
print(var_regression_error_ols)

std_dev_xb_ols = np.sqrt(np.diag(var_regression_error_ols * np.linalg.inv(W_b.transpose().dot(W_b))))
print("standard deviation of xb using OLS:")
print(std_dev_xb_ols)

pct_std_dev_xb_ols = std_dev_xb_ols / np.abs(xb_ols)
print("percentage of standard deviation of xb using OLS: ")
print(pct_std_dev_xb_ols)

variance of regression error using OLS:
0.00029406210815391344
standard deviation of xb using OLS:
[2.13065943e-06 7.77031141e-07 7.70964624e-07 1.19162010e-05
 1.18044808e-05 1.47165887e-05 2.58875236e-05 2.70527243e-06
 2.62807375e-06 2.61597920e-06 2.14457053e-05 2.48666125e-05
 3.41628215e-05 1.63086485e-05 1.36844101e-05 2.56460143e-05
 1.92633987e-05 3.86445381e-05 3.70108273e-05 3.88950367e-05
 3.20612698e-05 2.88127010e-05 3.02852537e-04 8.10187198e-04
 1.52278583e-04 3.21840104e-05 2.53185943e-03 2.73535714e-05
 2.46117949e-05 1.02659910e-04 2.63325732e-04 3.17097202e-04
 2.63551171e-04 3.55116336e-05 3.51681789e-04 2.63041936e-04
 4.16153528e-05 9.93450021e-05 6.32080953e-05 5.08550980e-04
 2.62237281e-04 8.12881681e-04 8.39408031e-04 5.31400679e-04
 2.35580722e-05 9.88154874e-05 1.05351783e-04 3.85986432e-04
 1.39755005e-04 2.62696300e-04 3.24876403e-04 4.58346937e-04
 3.26487179e-03 3.39236538e-04 1.42025436e-04 1.87593285e-04
 1.80192721e-04 1.50768580e-04 1.42375518e-04 2

### Plot measured torque and predicted torque

In [18]:
plot_data_num = int(1 / wf * trajectory_sampling_rate)
plot_meas_pred_tau(t_cut[:plot_data_num], tau_f[:plot_data_num, :], tau_p[:plot_data_num, :])

## Weighted Least Square (WLS)

In [19]:
np.sqrt(np.linalg.norm(tau_f - tau_p, axis=0)/(tau_f.shape[0] - base_param_num))

array([0.02260302, 0.04156412, 0.03030228, 0.0196861 , 0.01042662,
       0.01210039, 0.0033314 ])

In [20]:
weight = np.max(tau_f, axis=0) - np.min(tau_f, axis=0)
weight
weight[1] /= 1
weight[2] /= 1
weight[3] /= 1
weight

array([3.60738506, 4.38921693, 3.05917211, 0.42470381, 0.17455165,
       0.10147109, 0.00596803])

In [21]:
W_b.shape, tau_s.shape

In [22]:
# repeat the weight to generate a large vecoter for all the data
weights = 1.0/np.tile(weight, W_b.shape[0]/weight.shape[0])

In [23]:
weights.shape

In [24]:
W_b_wls = np.multiply(W_b, np.asmatrix(weights).transpose())
tau_s_wls = np.multiply(tau_s, weights)

In [25]:
xb_wls = np.linalg.lstsq(W_b_wls, tau_s_wls)[0]

/home/yan/anaconda2/envs/dynamics/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """Entry point for launching an IPython kernel.


In [26]:
sympy.Matrix.hstack(sympy.Matrix(robot_model.base_param), sympy.Matrix(xb_wls))

⎡                                                                  1.0⋅L7xy   
⎢                                                                             
⎢                                                                  1.0⋅L7xz   
⎢                                                                             
⎢                                                                  1.0⋅L7yz   
⎢                                                                             
⎢                                                                  1.0⋅L6xy   
⎢                                                                             
⎢                                                                  1.0⋅L6xz   
⎢                                                                             
⎢                                                                  1.0⋅L6yz   
⎢                                                                             
⎢                                                   

In [27]:
tau_p_wls = np.zeros(tau_f.shape)
tau_ps_wls = W_b.dot(xb_wls)
for i in range(dof):
    tau_p_wls[:, i] = tau_ps_wls[i::dof]

In [28]:
plot_meas_pred_tau(t_cut[:plot_data_num], tau_f[:plot_data_num, :], tau_p_wls[:plot_data_num, :])

In [29]:
np.linalg.norm(tau_f[:plot_data_num, :] - tau_p_wls[:plot_data_num, :], axis=0)\
/ np.linalg.norm(tau_f[:plot_data_num, :], axis=0)

array([0.06793809, 0.26859411, 0.16367809, 0.34087756, 0.2453117 ,
       0.21977706, 0.21779594])

## Convex optimization

### Generate regressor matrix for barycentric parameters

In [149]:
from identification import SDPOpt

bary_param_num = len(robot_model.bary_param)
H_func = robot_model.H_func
W, tau_s = gen_regressor(bary_param_num, H_func, q_f, dq_f, ddq_f, tau_f)

In [150]:
W_cvx = np.multiply(W, np.asmatrix(weights).transpose())
tau_s_cvx = np.multiply(tau_s, weights)

In [151]:
len(robot_model.std_param)

| min mass | max mass | min $r_x$ | max $r_x$ | min $r_y$ | max $r_y$ | min $r_z$ | max $r_z$ | max $F_c$ | max $F_v$ | max $F_o$ |
|----------|----------|-----------|-----------|-----------|-----------|-----------|-----------|-----------|--------------|-----------|

In [173]:
sdp_constraints = []
if model_name == 'mtm':
    sdp_constraints = [(0.5, 10, -0.1, 0.1, -0.1, 0.1, -0.1, 0.3, 0.2, 0.2, 0.2),
                      (0.3, 5, -0.1, 0.2, -0.05, 0.05, -0.05, 0.05, 0.2, 0.2, 0.2),
                      (0.1, 3, -0.0, 0.3, -0.02, 0.02, -0.01, 0.01, 0.2, 0.2, 0.2),
                      (0.1, 1, -0.01, 0.01, -0.0, 0.1, -0.1, 0.0, 0.2, 0.2, 0.1),
                      (0.02, 0.3, -0.01, 0.01, -0.05, 0.0, -0.05, 0.0, 0.1, 0.1, 0.1),
                      (0.02, 0.2, -0.005, 0.005, -0.00, 0.05, -0.05, 0.0, 0.1, 0.1, 0.1),
                      (0.01, 0.1, -0.000, 0.000, -0.00, 0.00, -0.00, 0.05, 0.1, 0.1, 0.1),
                      (0.3, 5, -0.10, 0.05, -0.05, 0.05, -0.05, 0.05, 0.1, 0.2, 0.2),
                      (0.05, 0.5, 0.10, 0.15, -0.00, 0.00, 0.025, 0.03, 0.1, 0.1, 0.1),
                      (0.0, 0, 0.00, 0.00, -0.00, 0.00, 0.000, 0.00, 0.1, 0.1, 0.0)]
elif model_name == 'mtm_3links':
    sdp_constraints = [(0.5, 10, -0.1, 0.1, -0.1, 0.1, -0.1, 0.3, 0.5, 0.5, 0.5),
                      (0.3, 5, -0.1, 0.2, -0.05, 0.05, -0.05, 0.05, 0.5, 0.5, 0.5),
                      (0.1, 3, -0.0, 0.3, -0.02, 0.02, -0.01, 0.01, 0.1, 0.5, 0.5),
                      (0.1, 1, -0.01, 0.01, -0.0, 0.1, -0.1, 0.0, 0.1, 0.1, 0.1),
                      (0.02, 0.3, -0.01, 0.01, -0.05, 0.0, -0.05, 0.0, 0.1, 0.1, 0.1),
                      (0.02, 0.2, -0.005, 0.005, -0.00, 0.05, -0.05, 0.0, 0.1, 0.1, 0.1),
                      (0.01, 0.1, -0.000, 0.000, -0.00, 0.00, -0.00, 0.05, 0.1, 0.1, 0.1),
                      (0.3, 5, -0.10, 0.05, -0.05, 0.05, -0.05, 0.05, 0.5, 0.5, 0.5),
                      (0.3, 5, 0.10, 0.15, -0.00, 0.00, 0.025, 0.03, 0.1, 0.1, 0.1),
                      (0.3, 5, 0.10, 0.15, -0.00, 0.00, 0.025, 0.03, 0.1, 0.1, 0.1)]
else:
    raise Exception("No robot name matches " + model_name)

In [174]:


sdp_opt_std = SDPOpt(W_cvx, tau_s_cvx, robot_model, sdp_constraints)
# sdp_opt_std = SDPOpt(W, tau_s, robot_model, sdp_constraints)
%time sdp_opt_std.solve()

Regressor shape: (102123, 120)
Regressand shape: (102123,)
Creating variables...
Creating optimization objective...
Creating constraints...
Solving problem...
----------------------------------------------------------------------------
	SCS v1.2.6 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2016
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 4268296, CG tol ~ 1/iter^(2.00)
eps = 1.00e-03, alpha = 1.50, max_iters = 10000, normalize = 1, scale = 1.00
Variables n = 121, constraints m = 102447
Cones:	linear vars: 133
	soc vars: 102125, soc blks: 1
	sd vars: 189, sd blks: 9
Setup time: 1.62e-01s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0|      inf       inf      -nan      -inf       inf       inf  3.50

  9400| 3.10e-01  1.06e-02  5.74e-04  1.25e+02  1.26e+02  5.87e-14  1.63e+03 
  9500| 3.10e-01  1.06e-02  5.65e-04  1.27e+02  1.27e+02  5.98e-14  1.65e+03 
  9600| 3.09e-01  1.00e-02  5.54e-04  1.28e+02  1.28e+02  6.08e-14  1.67e+03 
  9700| 3.09e-01  1.02e-02  5.43e-04  1.29e+02  1.29e+02  6.19e-14  1.69e+03 
  9800| 3.08e-01  1.05e-02  5.35e-04  1.30e+02  1.30e+02  6.30e-14  1.70e+03 
  9900| 3.08e-01  9.87e-03  5.24e-04  1.31e+02  1.32e+02  6.40e-14  1.72e+03 
 10000| 3.08e-01  9.73e-03  5.15e-04  1.33e+02  1.33e+02  6.51e-14  1.74e+03 
----------------------------------------------------------------------------
Status: Solved/Inaccurate
Hit max_iters, solution may be inaccurate
Timing: Solve time: 1.74e+03s
	Lin-sys: avg # CG iterations: 35.15, avg solve time: 1.72e-01s
	Cones: avg projection time: 2.59e-04s
----------------------------------------------------------------------------
Error metrics:
dist(s, K) = 6.4829e-09, dist(y, K*) = 2.7388e-09, s'y/|s||y| = 2.0676e-11
|Ax + s -

## Compare measured torque and predicted torque

### Generate predicted torque

In [175]:
tau_sdp_p = np.zeros(tau_f.shape)
tau_sdp_ps = W.dot(sdp_opt_std.x_result)
for i in range(dof):
    tau_sdp_p[:, i] = tau_sdp_ps[i::dof]

### Plot measured torque and predicted torque

In [200]:
plot_data_num = int(1 / wf * trajectory_sampling_rate)
plot_meas_pred_tau(t_cut[:plot_data_num] - t_cut[0],
                   tau_f[:plot_data_num, :],
                   tau_sdp_p[:plot_data_num, :])
# plot_meas_pred_tau(t_cut, tau_f, tau_sdp_p)

In [196]:
sympy.Matrix.hstack(sympy.Matrix(robot_model.bary_param), sympy.Matrix(sdp_opt_std.x_result))

⎡L1xx    0.986192165056305  ⎤
⎢                           ⎥
⎢L1xy  -1.38516767370781e-14⎥
⎢                           ⎥
⎢L1xz  -7.76343556708024e-27⎥
⎢                           ⎥
⎢L1yy    0.986192165056345  ⎥
⎢                           ⎥
⎢L1yz  -2.12133368993413e-16⎥
⎢                           ⎥
⎢L1zz   5.58233934249789e-5 ⎥
⎢                           ⎥
⎢l1x   2.40588966703881e-27 ⎥
⎢                           ⎥
⎢l1y   4.76358117877349e-17 ⎥
⎢                           ⎥
⎢l1z     0.144175828054554  ⎥
⎢                           ⎥
⎢ m₁     1.79307211904898   ⎥
⎢                           ⎥
⎢Fc₁    0.0419135562259835  ⎥
⎢                           ⎥
⎢Fv₁    0.0497210098280573  ⎥
⎢                           ⎥
⎢Fo₁    -0.0698865408680767 ⎥
⎢                           ⎥
⎢L2xx   0.0194654973664062  ⎥
⎢                           ⎥
⎢L2xy   0.00876972571924054 ⎥
⎢                           ⎥
⎢L2xz   0.00593865075240333 ⎥
⎢                           ⎥
⎢L2yy   0.0330003696264969  ⎥
⎢         

In [178]:
from identification import barycentric2standard_params

In [179]:
sdp_opt_std.x_result

array([ 9.86192165e-01, -1.38516767e-14, -7.76343557e-27,  9.86192165e-01,
       -2.12133369e-16,  5.58233934e-05,  2.40588967e-27,  4.76358118e-17,
        1.44175828e-01,  1.79307212e+00,  4.19135562e-02,  4.97210098e-02,
       -6.98865409e-02,  1.94654974e-02,  8.76972572e-03,  5.93865075e-03,
        3.30003696e-02,  9.67284032e-04,  5.04838084e-02, -4.02232774e-01,
        6.80708808e-02,  6.43105023e-04,  5.00027870e+00,  1.09124988e-01,
        1.99956998e-01, -2.00003749e-01,  6.19401340e-04,  8.10870769e-04,
        6.31621227e-05,  1.18055051e-03, -3.76610295e-05,  1.71030714e-03,
        4.07898714e-02, -2.79045453e-02, -2.19072695e-03,  1.41404147e+00,
        1.00110498e-03,  9.95052980e-04,  2.00008574e-01,  2.02184077e-02,
        3.45713217e-03,  1.34006236e-03,  1.24674614e-02,  2.77597880e-03,
        1.57090952e-03, -8.13941900e-04,  1.04485762e-02, -1.29595950e-02,
        1.33525052e-01,  4.08774176e-02,  1.22874520e-02, -1.04736499e-04,
        7.97410096e-04, -

In [180]:
x_std = barycentric2standard_params(sdp_opt_std.x_result, robot_model)

In [181]:
sympy.Matrix.hstack(sympy.Matrix(robot_model.std_param), sympy.Matrix(x_std))

⎡I1xx    0.974599396882406  ⎤
⎢                           ⎥
⎢I1xy  -1.38516767370781e-14⎥
⎢                           ⎥
⎢I1xz  -7.56998482337162e-27⎥
⎢                           ⎥
⎢I1yy    0.974599396882446  ⎥
⎢                           ⎥
⎢I1yz  -2.08303108884751e-16⎥
⎢                           ⎥
⎢I1zz   5.58233934249789e-5 ⎥
⎢                           ⎥
⎢r1x   1.34176960395484e-27 ⎥
⎢                           ⎥
⎢r1y   2.65665899779873e-17 ⎥
⎢                           ⎥
⎢r1z    0.0804071551405432  ⎥
⎢                           ⎥
⎢ m₁     1.79307211904898   ⎥
⎢                           ⎥
⎢Fc₁    0.0419135562259835  ⎥
⎢                           ⎥
⎢Fv₁    0.0497210098280573  ⎥
⎢                           ⎥
⎢Fo₁    -0.0698865408680767 ⎥
⎢                           ⎥
⎢I2xx   0.0185387373447571  ⎥
⎢                           ⎥
⎢I2xy   0.00329396309376292 ⎥
⎢                           ⎥
⎢I2xz   0.00588691805246235 ⎥
⎢                           ⎥
⎢I2yy  0.000643849538560864 ⎥
⎢         

In [182]:
sdp_constraints